In [1]:
import torch
print(torch.cuda.is_available())
import warnings
warnings.filterwarnings('ignore')

True


In [2]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

#AutoModelForQuestionAnswering: A class to load pre-trained models for question-answering tasks.
#AutoTokenizer: A class to load pre-trained tokenizers for models.
#pipeline: A utility function to create an end-to-end pipeline for various NLP tasks.

In [3]:
model_name = "deepset/roberta-base-squad2" 

#Pre-trained model you are using. deepset/roberta-base-squad2 is a model fine-tuned for the SQuAD2.0 dataset, designed for question-answering tasks

In [4]:
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name, device=0)

#creating a question-answering pipeline using the specified model and tokenizer.

In [5]:
QA_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}

#question: The question you want to ask.
#context: The passage or text from which the answer will be extracted.

In [11]:
res = nlp(QA_input)

#Applies the pipeline to the input dictionary, performing the question-answering task

print(res['answer'])

gives freedom to the user


In [7]:
QA_input2 = {
    'question': 'What are some examples of traditional Indian painting styles?',
    'context': """Indian art and culture have a rich and diverse history that spans thousands of years.
    One of the most prominent features of Indian art is its intricate and symbolic nature, seen in forms 
    ranging from ancient cave paintings in Ajanta and Ellora to the detailed sculptures of Khajuraho.
    Indian classical dance forms like Bharatanatyam, Kathak, and Odissi reflect the deep connection between art, 
    spirituality, and storytelling, often drawing on themes from Hindu mythology. Similarly, Indian music, 
    both classical (like Hindustani and Carnatic) and folk, plays a significant role in cultural expression, 
    often accompanying religious and social ceremonies. The traditional art of Indian painting, 
    including styles like Madhubani, Warli, and Pattachitra, uses vibrant colors and motifs that depict nature, 
    mythology, and daily life. Indian architecture, from the grandeur of the Taj Mahal to the intricate carvings of South Indian temples, 
    showcases the skill and aesthetic sensibilities of Indian artisans. 
    Indian culture is not just a celebration of visual arts but also includes a rich tradition of crafts such as pottery, 
    weaving, and jewelry making, which have been passed down through generations."""
}


In [10]:
res2 = nlp(QA_input2)
print(res2['answer'])

Madhubani, Warli, and Pattachitra
